## **1. Preprocesamiento**

In [1]:
import os
import shutil
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')

# Directorios
ruta_base = '/Users/alexperez/Documents/GitHub/DM1/TextMining/dataset/Procesos'
ruta_clean = '/Users/alexperez/Documents/GitHub/DM1/TextMining/dataset/Procesos_Clean'

# Función para gestionar la carpeta clean
def gestionar_carpeta_clean(ruta_clean):
    try:
        if os.path.exists(ruta_clean):
            print(f'La carpeta {ruta_clean} ya existe, eliminando y recreando...')
            shutil.rmtree(ruta_clean)
        os.makedirs(ruta_clean)
        print(f'Carpeta {ruta_clean} creada nuevamente.')
    except Exception as e:
        print(f'Error al gestionar la carpeta {ruta_clean}: {e}')

# Lista de siglas irrelevantes o ruido
siglas_inecesarias = ['etc']
def limpiar_texto(texto):
    # Convertir a minúsculas
    texto = texto.lower()
    
    # Remover caracteres especiales, excepto letras, espacios y tildes
    texto = re.sub(r'[^a-záéíóúÁÉÍÓÚñÑ\s]', '', texto)
    
    # Tokenizar el texto
    palabras = word_tokenize(texto)
    
    # Eliminar stopwords en español
    stop_words = set(stopwords.words('spanish'))
    palabras_limpias = [palabra for palabra in palabras if palabra not in stop_words]
    
    # Eliminar palabras de menos de 3 caracteres
    palabras_limpias = [palabra for palabra in palabras_limpias if len(palabra) >= 3]
    
    # Eliminar secuencias de letras repetidas (ej. "aaa", "bbb")
    palabras_limpias = [palabra for palabra in palabras_limpias if not re.fullmatch(r'(.)\1+', palabra)]
    
    # Eliminar siglas irrelevantes
    palabras_limpias = [palabra for palabra in palabras_limpias if palabra not in siglas_inecesarias]
    
    # Unir las palabras limpias
    texto_limpio = ' '.join(palabras_limpias)
    
    return texto_limpio

# Función para procesar los archivos en cada carpeta
def procesar_archivos(carpeta_raiz, ruta_relativa):
    ruta_nueva_carpeta = os.path.join(ruta_clean, ruta_relativa)
    
    # Crear la nueva carpeta si no existe
    if not os.path.exists(ruta_nueva_carpeta):
        os.makedirs(ruta_nueva_carpeta)

    # Procesar cada archivo dentro de la carpeta
    for archivo in os.listdir(carpeta_raiz):
        if archivo.endswith('.txt'):  # Solo procesar archivos .txt
            ruta_archivo = os.path.join(carpeta_raiz, archivo)
            ruta_archivo_clean = os.path.join(ruta_nueva_carpeta, archivo)
            
            try:
                # Leer el contenido del archivo
                with open(ruta_archivo, 'r', encoding='utf-8') as file:
                    contenido = file.read()
                
                # Limpiar y normalizar el contenido
                contenido_limpio = limpiar_texto(contenido)
                
                # Guardar el archivo limpio en la nueva carpeta
                with open(ruta_archivo_clean, 'w', encoding='utf-8') as file_clean:
                    file_clean.write(contenido_limpio)

                # Imprimir el nombre del archivo procesado y su carpeta
                print(f'Procesado: {archivo} en la carpeta {ruta_relativa}')
            except Exception as e:
                print(f'Error al procesar el archivo {archivo}: {e}')

# Función principal para recorrer todas las carpetas
def limpiar_procesos(ruta_base):
    for carpeta_raiz, _, _ in os.walk(ruta_base):
        # Obtener la ruta relativa para recrear la misma estructura de carpetas
        ruta_relativa = os.path.relpath(carpeta_raiz, ruta_base)
        procesar_archivos(carpeta_raiz, ruta_relativa)

# Ejecutar todo el proceso
def ejecutar_limpieza():
    # Gestionar la carpeta clean
    gestionar_carpeta_clean(ruta_clean)
    
    # Limpiar los archivos
    limpiar_procesos(ruta_base)
    
    print("Limpieza de archivos completada.")

# Ejecutar el proceso de limpieza
ejecutar_limpieza()

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


[nltk_data] Downloading package punkt to /Users/alexperez/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/alexperez/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


La carpeta /Users/alexperez/Documents/GitHub/DM1/TextMining/dataset/Procesos_Clean ya existe, eliminando y recreando...
Carpeta /Users/alexperez/Documents/GitHub/DM1/TextMining/dataset/Procesos_Clean creada nuevamente.
Procesado: 0_indice_de_subasta_inversa_electronica1.txt en la carpeta 1718396
Procesado: 2_condiciones_generales_del_pliego_de_subasta_inversa_electronica_bienes_y_servicios.txt en la carpeta 1718396
Procesado: 1_condiciones_particulares_del_pliego_de_subasta_inversa_electronica1.txt en la carpeta 1718396
Procesado: formulario-unico-oferta-subasta-inversa-bys.txt en la carpeta 1718396
Procesado: condiciones_particulares_del_contrato_de_subasta_inversa_electronica.txt en la carpeta 1718396
Procesado: 5_condiciones_generales_del_contrato_de_subasta_inversa_electronica_bienes_y_servicios.txt en la carpeta 1718396
Procesado: tdr_sun_oracle-signed-signed-signed.txt en la carpeta 1718396
Procesado: 3_formulario_de_compromiso_de_asociacion_o_consorcio.txt en la carpeta 1718396


## **2. Vectores TF-IDF**

In [3]:
import pandas as pd
import os

# Cargar archivos CSV
process_info = pd.read_csv('C:/USFQ/9_Semestre/DataMining/TextMining/process_info.csv')
preguntas_y_aclaraciones = pd.read_csv('C:/USFQ/9_Semestre/DataMining/TextMining/preguntas_y_aclaraciones.csv')
sample_files = pd.read_csv('C:/USFQ/9_Semestre/DataMining/TextMining/sample_files.csv')

# Verificar que los datos se cargaron correctamente
print(process_info.head())
print(preguntas_y_aclaraciones.head())
print(sample_files.head())

   sl_contract_id                                         sd_entidad  \
0         1752147  GOBIERNO AUTONOMO DESCENTRALIZADO MUNICIPAL DE...   
1         1689929  CENTRO CLINICO QUIRURGICO AMBULATORIO HOSPITAL...   
2         1737580  EMPRESA ELÉCTRICA PÚBLICA ESTRATÉGICA CORPORAC...   
3         1699371                         HOSPITAL BÁSICO-ESMERALDAS   
4         1712786  DIRECCION DISTRITAL 09D04 - FEBRES CORDERO - S...   

                                sd_objeto_de_proceso  sie_ic_promedio  
0  ADQUISICION DE UNA MAQUINA EXCAVADORA DE ORUGA...             0.86  
1  ADQUISICIÓN DEL PRIMER REQUERIMIENTO DE FÁRMAC...             0.73  
2         CORP ADQUISICION DE SELLOS DE SEGURIDAD GC             0.63  
3  ADQUISICION DE MEDICAMENTOS CUYO GRUPO TERAPÉU...             0.73  
4  CONTRATACIÓN DEL SERVICIO DE VIGILANCIA Y SEGU...             0.55  
   sl_contract_id  pregunta_id  \
0         1728041       326428   
1         1706638       332456   
2         1706638       332449   

In [17]:
import os
from sklearn.feature_extraction.text import TfidfVectorizer

def textos_individuales_col1(sample_files):
    textos_por_documento = {}  # Diccionario para almacenar documentos por contract_id

    # Procesar los archivos de texto de los procesos (cada archivo es un documento independiente)
    for index, row in sample_files.iterrows():
        sl_contract_id = str(row['sl_contract_id'])
        file_name = row['file_name']  # Nombre del archivo

        # Ajustar la ruta para acceder a la carpeta correcta
        file_path = f"C:/USFQ/9_Semestre/DataMining/TextMining/Procesos_Clean/{sl_contract_id}/{file_name}.txt"

        try:
            # Verificar si el archivo existe
            if os.path.exists(file_path):
                # Leer el contenido del archivo
                with open(file_path, 'r', encoding='utf-8') as file:
                    contenido = file.read()
                # Agregar el contenido al diccionario de textos para COL1 (sin concatenar)
                if sl_contract_id in textos_por_documento:
                    textos_por_documento[sl_contract_id].append(contenido)
                else:
                    textos_por_documento[sl_contract_id] = [contenido]
            else:
                print(f"Archivo no encontrado: {file_path}")

        except Exception as e:
            print(f"Error al leer el archivo {file_path}: {e}")

    # Retornar el diccionario donde cada proceso tiene su lista de documentos
    return textos_por_documento

# Llamar a la función para obtener los textos individuales (colección COL1)
textos_individuales_corpus_col1 = textos_individuales_col1(sample_files)

# Descomponer los documentos en una lista donde cada documento es independiente, pero guardamos el proceso al que pertenecen
documentos_col1 = []
procesos_col1 = []

for proceso, documentos in textos_individuales_corpus_col1.items():
    for documento in documentos:
        documentos_col1.append(documento)  # Agregar documento individual a la lista
        procesos_col1.append(proceso)  # Mantener el registro del proceso al que pertenece

# Vectorizar los documentos individuales de COL1
vectorizer_col1 = TfidfVectorizer()
X_tfidf_col1 = vectorizer_col1.fit_transform(documentos_col1)

# Mostrar la forma de la matriz TF-IDF resultante para COL1
print(f"Matriz TF-IDF de COL1 con forma: {X_tfidf_col1.shape}")

# Verificar que los procesos están alineados con los documentos
for i in range(20):  # Mostrar los primeros 5 documentos y sus procesos asociados
    print(f"Documento {i+1} pertenece al proceso {procesos_col1[i]}")

# Verificar las dimensiones del diccionario de textos individuales
print("Número de procesos en COL1:", len(textos_individuales_corpus_col1))

Matriz TF-IDF de COL1 con forma: (828, 28546)
Documento 1 pertenece al proceso 1721524
Documento 2 pertenece al proceso 1721524
Documento 3 pertenece al proceso 1721524
Documento 4 pertenece al proceso 1721524
Documento 5 pertenece al proceso 1721524
Documento 6 pertenece al proceso 1721524
Documento 7 pertenece al proceso 1721524
Documento 8 pertenece al proceso 1721524
Documento 9 pertenece al proceso 1721524
Documento 10 pertenece al proceso 1721524
Documento 11 pertenece al proceso 1699371
Documento 12 pertenece al proceso 1699371
Documento 13 pertenece al proceso 1699371
Documento 14 pertenece al proceso 1699371
Documento 15 pertenece al proceso 1699371
Documento 16 pertenece al proceso 1699371
Documento 17 pertenece al proceso 1699371
Documento 18 pertenece al proceso 1745524
Documento 19 pertenece al proceso 1745524
Documento 20 pertenece al proceso 1745524
Número de procesos en COL1: 107


In [19]:

# Función para concatenar textos de los archivos de procesos y preguntas/respuestas
def concatenar_textos(sample_files, preguntas_y_aclaraciones):
    textos_por_proceso = {}

    # Procesar los archivos de texto de los procesos
    for index, row in sample_files.iterrows():
        sl_contract_id = str(row['sl_contract_id'])
        file_name = row['file_name']  # Ajustar para obtener el nombre del archivo

        # Ajustar la ruta para acceder a la carpeta correcta
        file_path = f"C:/USFQ/9_Semestre/DataMining/TextMining/Procesos_Clean/{sl_contract_id}/{file_name}.txt"

        try:
            # Leer el contenido del archivo
            with open(file_path, 'r', encoding='utf-8') as file:
                contenido = file.read()
                contenido_limpio = limpiar_texto(contenido)  # Aplicar la función de limpieza

            # Concatenar contenido en caso de que ya existan otros archivos para ese proceso
            if sl_contract_id in textos_por_proceso:
                textos_por_proceso[sl_contract_id] += ' ' + contenido_limpio
            else:
                textos_por_proceso[sl_contract_id] = contenido_limpio
        except Exception as e:
            print(f"Error al leer el archivo {file_path}: {e}")

    # Procesar las preguntas y respuestas y concatenarlas al texto del proceso
    for index, row in preguntas_y_aclaraciones.iterrows():
        sl_contract_id = str(row['sl_contract_id'])

        # Convertir valores nulos a cadenas vacías antes de concatenar
        pregunta_aclaracion = str(row['pregunta_aclaracion']) if not pd.isna(row['pregunta_aclaracion']) else ''
        respuesta_aclaracion = str(row['respuesta_aclaracion']) if not pd.isna(row['respuesta_aclaracion']) else ''

        pregunta_respuesta = pregunta_aclaracion + ' ' + respuesta_aclaracion
        pregunta_respuesta_limpia = limpiar_texto(pregunta_respuesta)

        if sl_contract_id in textos_por_proceso:
            textos_por_proceso[sl_contract_id] += ' ' + pregunta_respuesta_limpia
        else:
            textos_por_proceso[sl_contract_id] = pregunta_respuesta_limpia

    return textos_por_proceso

# Llamar a la función para concatenar los textos
textos_por_proceso = concatenar_textos(sample_files, preguntas_y_aclaraciones)

from sklearn.feature_extraction.text import TfidfVectorizer

# Crear una lista con los textos preprocesados y concatenados por proceso
corpus = list(textos_por_proceso.values())
contract_ids = list(textos_por_proceso.keys())

# Crear el vectorizador TF-IDF y ajustar con el contenido completo
vectorizer_col2 = TfidfVectorizer()
X_tfidf_col2 = vectorizer_col2.fit(corpus)  # Aquí estamos haciendo fit con todo el corpus

# Transformar el corpus usando el vectorizador ajustado
X_tfidf_col2 = vectorizer_col2.transform(corpus)

# Mostrar la forma de la matriz TF-IDF resultante para COL2
print(f"Matriz TF-IDF de COL2 con forma: {X_tfidf_col2.shape}")

# Mostrar las dimensiones del dicccionario de la colección COL2
print("Número de procesos en COL2:", len(textos_por_proceso))

for i in range(20):  # Mostrar los primeros 5 documentos y sus procesos asociados
    print(f"Documento {i+1} pertenece al proceso {contract_ids[i]}")

Matriz TF-IDF de COL2 con forma: (107, 30981)
Número de procesos en COL2: 107
Documento 1 pertenece al proceso 1721524
Documento 2 pertenece al proceso 1699371
Documento 3 pertenece al proceso 1745524
Documento 4 pertenece al proceso 1758687
Documento 5 pertenece al proceso 1704225
Documento 6 pertenece al proceso 1747722
Documento 7 pertenece al proceso 1686226
Documento 8 pertenece al proceso 1709477
Documento 9 pertenece al proceso 1755826
Documento 10 pertenece al proceso 1728041
Documento 11 pertenece al proceso 1682008
Documento 12 pertenece al proceso 1743489
Documento 13 pertenece al proceso 1709875
Documento 14 pertenece al proceso 1706638
Documento 15 pertenece al proceso 1716637
Documento 16 pertenece al proceso 1730092
Documento 17 pertenece al proceso 1736958
Documento 18 pertenece al proceso 1718330
Documento 19 pertenece al proceso 1750232
Documento 20 pertenece al proceso 1682777


## **3. Cálculo de TF, DF, IDF y TF-IDF para COL1 y COL2**

### A. Cálculo para COL2

In [24]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

# Ya tienes el TfidfVectorizer para COL1
# Mostramos los nombres de los términos (palabras) del vocabulario de COL1
terms_col1 = vectorizer_col1.get_feature_names_out()

# Obtenemos la matriz de cuenta de términos (TF y DF) usando CountVectorizer
count_vectorizer_col1 = CountVectorizer()
X_count_col1 = count_vectorizer_col1.fit_transform(textos_por_proceso.values())

# TF: Frecuencia de términos en todos los documentos de COL1
tf_col1 = np.sum(X_count_col1.toarray(), axis=0)

# DF: Número de documentos en los que aparece cada término en COL1
df_col1 = np.sum(X_count_col1.toarray() > 0, axis=0)

# IDF: Ya está calculado en el TfidfVectorizer
idf_col1 = vectorizer_col1.idf_

# Mostrar resultados para los primeros 15 términos
for i in range(15):
    term = terms_col1[i]
    print(f"Término: {term}")
    print(f"TF (Frecuencia de Término) en COL1: {tf_col1[i]}")
    print(f"DF (Frecuencia de Documento) en COL1: {df_col1[i]}")
    print(f"IDF (Frecuencia Inversa de Documento) en COL1: {idf_col1[i]}")
    
    # Mostrar el valor de TF-IDF del término en los primeros documentos
    #print(f"TF-IDF en los primeros documentos de COL1: {X_tfidf_col1[:, i].toarray().flatten()[:20]}")
    print(f"TF-IDF en los primeros documentos de COL1: {X_tfidf_col1[:, i].toarray().flatten().shape}")
    print("-" * 40)

Término: aabsly
TF (Frecuencia de Término) en COL1: 1
DF (Frecuencia de Documento) en COL1: 1
IDF (Frecuencia Inversa de Documento) en COL1: 7.02707297457535
TF-IDF en los primeros documentos de COL1: (828,)
----------------------------------------
Término: aac
TF (Frecuencia de Término) en COL1: 4
DF (Frecuencia de Documento) en COL1: 2
IDF (Frecuencia Inversa de Documento) en COL1: 6.621607866467185
TF-IDF en los primeros documentos de COL1: (828,)
----------------------------------------
Término: aad
TF (Frecuencia de Término) en COL1: 2
DF (Frecuencia de Documento) en COL1: 1
IDF (Frecuencia Inversa de Documento) en COL1: 7.02707297457535
TF-IDF en los primeros documentos de COL1: (828,)
----------------------------------------
Término: aam
TF (Frecuencia de Término) en COL1: 1
DF (Frecuencia de Documento) en COL1: 1
IDF (Frecuencia Inversa de Documento) en COL1: 7.02707297457535
TF-IDF en los primeros documentos de COL1: (828,)
----------------------------------------
Término: aam

### B. Cálculo para COL2

NameError: name 'textos_por_proceso_col2' is not defined